In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np 

In [2]:
df = pd.read_excel('spreadsheet.xls', index_col=0)
df = df.iloc[0:-1, 0:-1]
df.head()

,1: Toy Story (1995),1210: Star Wars: Episode VI - Return of the Jedi (1983),356: Forrest Gump (1994),"318: Shawshank Redemption, The (1994)","593: Silence of the Lambs, The (1991)",3578: Gladiator (2000),260: Star Wars: Episode IV - A New Hope (1977),2028: Saving Private Ryan (1998),296: Pulp Fiction (1994),1259: Stand by Me (1986),2396: Shakespeare in Love (1998),2916: Total Recall (1990),780: Independence Day (ID4) (1996),541: Blade Runner (1982),1265: Groundhog Day (1993),"2571: Matrix, The (1999)",527: Schindler's List (1993),"2762: Sixth Sense, The (1999)",1198: Raiders of the Lost Ark (1981),34: Babe (1995)
User,,,,,,,,,,,,,,,,,,,,
755,2.0,5.0,2.0,NaN,4.0,4.0,1.0,2.0,NaN,3.0,2.0,NaN,5.0,2.0,5.0,4.0,2.0,5.0,NaN,NaN
5277,1.0,NaN,NaN,2.0,4.0,2.0,5.0,NaN,NaN,4.0,3.0,2.0,2.0,NaN,2.0,NaN,5.0,1.0,3.0,NaN
1577,NaN,NaN,NaN,5.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,4.0,4.0,1.0,1.0,2.0,3.0,1.0,3.0
4388,2.0,3.0,NaN,NaN,NaN,1.0,NaN,3.0,4.0,NaN,NaN,4.0,NaN,3.0,5.0,NaN,5.0,1.0,1.0,2.0
1202,NaN,3.0,4.0,1.0,4.0,1.0,4.0,4.0,NaN,1.0,5.0,1.0,NaN,4.0,NaN,3.0,5.0,5.0,NaN,NaN


In [3]:
df_values = torch.Tensor(df.values)

In [4]:
mean_user_rankings = torch.nanmean(df_values, dim=1, keepdim=True) 

In [5]:
df_values_normalized = df_values - mean_user_rankings

In [6]:
# fill nan values with 0

df_values = torch.nan_to_num(df_values, 0)
df_values_normalized = torch.nan_to_num(df_values_normalized, 0)

In [7]:
cos = torch.nn.CosineSimilarity(dim=1)

In [8]:
# cosine similarity for first element 

cos(df_values[:, 0].reshape(1, -1), df_values.t())

tensor([1.0000, 0.6450, 0.5805, 0.6674, 0.5702, 0.5879, 0.7474, 0.5346, 0.6678,
        0.4927, 0.3767, 0.6231, 0.6907, 0.3831, 0.6610, 0.5050, 0.4638, 0.4216,
        0.4668, 0.6181])

In [9]:
# cosine similarity for first element (normalized)
cos(df_values_normalized[:, 0].reshape(1, -1), df_values_normalized.t())

tensor([ 1.0000, -0.3778,  0.3558,  0.2160, -0.6037,  0.1392, -0.0503,  0.1928,
         0.2950, -0.2257,  0.1168,  0.1847, -0.3199,  0.0200, -0.1056, -0.4203,
        -0.1468, -0.2413, -0.2489,  0.5544])

In [10]:
# matrix version
def get_cosine_similarities(matrix: torch.Tensor, colnames=list[str]): 
    m_normalized = F.normalize(matrix, p=2, dim=0)
    cosine_similarities = torch.mm(m_normalized.t(), m_normalized)

    return pd.DataFrame(
        cosine_similarities.numpy(), 
        columns=colnames,
        index=colnames
    )

In [11]:
df_similiarity_raw = get_cosine_similarities(df_values, df.columns)
df_similiarity_raw.head()

,1: Toy Story (1995),1210: Star Wars: Episode VI - Return of the Jedi (1983),356: Forrest Gump (1994),"318: Shawshank Redemption, The (1994)","593: Silence of the Lambs, The (1991)",3578: Gladiator (2000),260: Star Wars: Episode IV - A New Hope (1977),2028: Saving Private Ryan (1998),296: Pulp Fiction (1994),1259: Stand by Me (1986),2396: Shakespeare in Love (1998),2916: Total Recall (1990),780: Independence Day (ID4) (1996),541: Blade Runner (1982),1265: Groundhog Day (1993),"2571: Matrix, The (1999)",527: Schindler's List (1993),"2762: Sixth Sense, The (1999)",1198: Raiders of the Lost Ark (1981),34: Babe (1995)
1: Toy Story (1995),1.000000,0.644995,0.580540,0.667424,0.570229,0.587852,0.747409,0.534579,0.667846,0.492659,0.376659,0.623056,0.690665,0.383067,0.661016,0.505010,0.463817,0.421637,0.466817,0.618070
1210: Star Wars: Episode VI - Return of the Jedi (1983),0.644995,1.000000,0.563029,0.456052,0.516566,0.483187,0.589805,0.408753,0.685662,0.534324,0.533429,0.391934,0.605856,0.515397,0.526953,0.535673,0.573529,0.565297,0.252604,0.511576
356: Forrest Gump (1994),0.580540,0.563029,1.000000,0.293041,0.381346,0.569209,0.595550,0.463003,0.399114,0.527926,0.647153,0.491498,0.498741,0.487713,0.298290,0.631039,0.320494,0.602943,0.288275,0.456849
"318: Shawshank Redemption, The (1994)",0.667424,0.456052,0.293041,1.000000,0.589000,0.212846,0.565577,0.598344,0.538219,0.340151,0.329203,0.332674,0.617366,0.531981,0.437319,0.255345,0.497511,0.459446,0.467347,0.542782
"593: Silence of the Lambs, The (1991)",0.570229,0.516566,0.381346,0.589000,1.000000,0.551612,0.682137,0.640590,0.400471,0.661958,0.484751,0.414499,0.738445,0.585662,0.673091,0.530856,0.757630,0.715565,0.702452,0.309159


In [12]:
df_similarity_normalized = get_cosine_similarities(df_values_normalized, df.columns)
df_similarity_normalized.head()

,1: Toy Story (1995),1210: Star Wars: Episode VI - Return of the Jedi (1983),356: Forrest Gump (1994),"318: Shawshank Redemption, The (1994)","593: Silence of the Lambs, The (1991)",3578: Gladiator (2000),260: Star Wars: Episode IV - A New Hope (1977),2028: Saving Private Ryan (1998),296: Pulp Fiction (1994),1259: Stand by Me (1986),2396: Shakespeare in Love (1998),2916: Total Recall (1990),780: Independence Day (ID4) (1996),541: Blade Runner (1982),1265: Groundhog Day (1993),"2571: Matrix, The (1999)",527: Schindler's List (1993),"2762: Sixth Sense, The (1999)",1198: Raiders of the Lost Ark (1981),34: Babe (1995)
1: Toy Story (1995),1.000000,-0.377771,0.355780,0.215975,-0.603708,0.139214,-0.050338,0.192799,0.295013,-0.225657,0.116755,0.184712,-0.319904,0.020007,-0.105584,-0.420331,-0.146784,-0.241281,-0.248857,0.554447
1210: Star Wars: Episode VI - Return of the Jedi (1983),-0.377771,1.000000,-0.504038,0.003587,-0.023219,-0.328703,-0.231496,-0.156332,0.013543,0.315185,0.155409,-0.003662,0.493912,-0.159882,-0.271489,0.435885,-0.250274,0.016441,0.038346,-0.322875
356: Forrest Gump (1994),0.355780,-0.504038,1.000000,-0.305232,0.023470,0.391564,0.100946,0.075201,-0.031832,0.194059,0.133824,-0.184231,-0.199001,-0.158860,-0.258312,-0.344463,0.271580,-0.120007,-0.076494,-0.240155
"318: Shawshank Redemption, The (1994)",0.215975,0.003587,-0.305232,1.000000,-0.186192,-0.001011,-0.194869,-0.276413,0.343615,-0.062224,-0.096727,-0.049894,0.135022,0.341229,-0.101284,-0.067329,-0.213264,-0.261482,-0.343766,0.323532
"593: Silence of the Lambs, The (1991)",-0.603708,-0.023219,0.023470,-0.186192,1.000000,0.088282,-0.286670,-0.299883,-0.346559,0.112863,0.089045,-0.276003,0.196369,0.025442,0.011303,0.180366,0.227778,0.401320,-0.010014,-0.459929


In [13]:
df_similarity_normalized.loc['1: Toy Story (1995)'].sort_values(ascending=False)

1: Toy Story (1995)                                        1.000000
34: Babe (1995)                                            0.554447
356: Forrest Gump (1994)                                   0.355780
296: Pulp Fiction (1994)                                   0.295013
318: Shawshank Redemption, The (1994)                      0.215975
2028: Saving Private Ryan (1998)                           0.192799
2916: Total Recall (1990)                                  0.184712
3578: Gladiator (2000)                                     0.139214
2396: Shakespeare in Love (1998)                           0.116755
541: Blade Runner (1982)                                   0.020007
260: Star Wars: Episode IV - A New Hope (1977)            -0.050338
1265: Groundhog Day (1993)                                -0.105584
527: Schindler's List (1993)                              -0.146784
1259: Stand by Me (1986)                                  -0.225657
2762: Sixth Sense, The (1999)                   

### Part 2: Recommendations for user 5277

In [14]:
def get_recommendations_for_user(df_similiarities: pd.DataFrame, user_id):
    r = df.loc[user_id] - df.loc[user_id].mean()
    numerator = df_similiarities.mul(r).sum(axis=1)
    denominator = df_similiarity_raw.mul(df.loc[user_id].notnull()).sum()

    sim = numerator / denominator
    return sim[sim.abs() < np.inf].sort_values(ascending=False)

In [15]:
get_recommendations_for_user(df_similiarity_raw, 5277)

527: Schindler's List (1993)                      0.140321
1259: Stand by Me (1986)                          0.115227
260: Star Wars: Episode IV - A New Hope (1977)    0.103436
593: Silence of the Lambs, The (1991)             0.081836
2396: Shakespeare in Love (1998)                  0.054473
1198: Raiders of the Lost Ark (1981)              0.034997
3578: Gladiator (2000)                            0.027714
318: Shawshank Redemption, The (1994)            -0.030309
2762: Sixth Sense, The (1999)                    -0.035030
2916: Total Recall (1990)                        -0.047632
1265: Groundhog Day (1993)                       -0.054462
780: Independence Day (ID4) (1996)               -0.065512
1: Toy Story (1995)                              -0.067933
dtype: float64

In [16]:
get_recommendations_for_user(df_similarity_normalized, 5277)

527: Schindler's List (1993)                      0.407745
260: Star Wars: Episode IV - A New Hope (1977)    0.359240
1259: Stand by Me (1986)                          0.272647
2396: Shakespeare in Love (1998)                  0.184724
593: Silence of the Lambs, The (1991)             0.146354
1198: Raiders of the Lost Ark (1981)              0.062140
3578: Gladiator (2000)                           -0.140670
1265: Groundhog Day (1993)                       -0.151645
780: Independence Day (ID4) (1996)               -0.190667
2916: Total Recall (1990)                        -0.192549
318: Shawshank Redemption, The (1994)            -0.204833
1: Toy Story (1995)                              -0.249615
2762: Sixth Sense, The (1999)                    -0.298948
dtype: float64

## Putting it all together

In [17]:
df.head()

,1: Toy Story (1995),1210: Star Wars: Episode VI - Return of the Jedi (1983),356: Forrest Gump (1994),"318: Shawshank Redemption, The (1994)","593: Silence of the Lambs, The (1991)",3578: Gladiator (2000),260: Star Wars: Episode IV - A New Hope (1977),2028: Saving Private Ryan (1998),296: Pulp Fiction (1994),1259: Stand by Me (1986),2396: Shakespeare in Love (1998),2916: Total Recall (1990),780: Independence Day (ID4) (1996),541: Blade Runner (1982),1265: Groundhog Day (1993),"2571: Matrix, The (1999)",527: Schindler's List (1993),"2762: Sixth Sense, The (1999)",1198: Raiders of the Lost Ark (1981),34: Babe (1995)
User,,,,,,,,,,,,,,,,,,,,
755,2.0,5.0,2.0,NaN,4.0,4.0,1.0,2.0,NaN,3.0,2.0,NaN,5.0,2.0,5.0,4.0,2.0,5.0,NaN,NaN
5277,1.0,NaN,NaN,2.0,4.0,2.0,5.0,NaN,NaN,4.0,3.0,2.0,2.0,NaN,2.0,NaN,5.0,1.0,3.0,NaN
1577,NaN,NaN,NaN,5.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,4.0,4.0,1.0,1.0,2.0,3.0,1.0,3.0
4388,2.0,3.0,NaN,NaN,NaN,1.0,NaN,3.0,4.0,NaN,NaN,4.0,NaN,3.0,5.0,NaN,5.0,1.0,1.0,2.0
1202,NaN,3.0,4.0,1.0,4.0,1.0,4.0,4.0,NaN,1.0,5.0,1.0,NaN,4.0,NaN,3.0,5.0,5.0,NaN,NaN


In [18]:
movies = df.columns.values
users = df.index.values
ratings = torch.Tensor(df.values)

In [19]:
def cosine_similarity(ratings, normalize=True):

    if normalize: 
        mean_user_rankings = torch.nanmean(ratings, dim=1, keepdim=True) 
        ratings = ratings - mean_user_rankings

    # fill nan values with 0

    ratings = torch.nan_to_num(ratings, 0)

    m_normalized = F.normalize(ratings, p=2, dim=0)
    cos = torch.mm(m_normalized.t(), m_normalized)

    return cos

In [20]:
def get_similar_movies(movie_id, normalize = True, topn = 5): 
    idx = np.where(movies == movie_id)
    values = cosine_similarity(ratings, normalize=normalize)[idx]
    return pd.Series(values.reshape(-1), index=movies).sort_values(ascending=False).iloc[1:topn+1]

In [21]:
get_similar_movies('1: Toy Story (1995)', normalize=False)

260: Star Wars: Episode IV - A New Hope (1977)    0.747409
780: Independence Day (ID4) (1996)                0.690665
296: Pulp Fiction (1994)                          0.667846
318: Shawshank Redemption, The (1994)             0.667424
1265: Groundhog Day (1993)                        0.661016
dtype: float32

In [22]:
def get_user_recommendations(user_id, normalize=True):
    idx = np.where(users == user_id)
    user_ratings = ratings[idx]
    user_ratings

    r = user_ratings - user_ratings.nanmean()

    cos = cosine_similarity(ratings, normalize=normalize)

    numerator = (cos * r).nansum(axis=1)
    denominator = (cos * ~torch.isnan(user_ratings)).sum(axis=0)

    recommendations = numerator / denominator
    
    s_recom = pd.Series(recommendations, index=movies)
    return s_recom[s_recom.abs() < np.inf].sort_values(ascending=False)

In [23]:
get_user_recommendations(5277, True)

2396: Shakespeare in Love (1998)                   40.263653
527: Schindler's List (1993)                       30.714518
2762: Sixth Sense, The (1999)                      28.413553
1265: Groundhog Day (1993)                         21.222315
3578: Gladiator (2000)                              8.278975
1198: Raiders of the Lost Ark (1981)               -2.250456
1: Toy Story (1995)                                -8.733700
318: Shawshank Redemption, The (1994)              -9.806293
593: Silence of the Lambs, The (1991)             -12.818223
1259: Stand by Me (1986)                          -15.013106
780: Independence Day (ID4) (1996)                -24.566065
260: Star Wars: Episode IV - A New Hope (1977)    -27.179283
2916: Total Recall (1990)                        -203.454453
dtype: float32

In [24]:
def get_user_recommendations(user_id, normalize=True): 
    r = df.loc[user_id] - df.loc[user_id].mean()
    numerator = df_similarity_normalized.mul(r).sum(axis=1)
    denominator = df_similarity_normalized.mul(df.loc[user_id].notnull()).sum()

    sim = numerator / denominator

    return sim[sim.abs() < np.inf].sort_values(ascending=False)


In [25]:
get_user_recommendations(5277).sort_values(ascending=False)

2396: Shakespeare in Love (1998)                   40.263679
527: Schindler's List (1993)                       30.714503
2762: Sixth Sense, The (1999)                      28.413506
1265: Groundhog Day (1993)                         21.222323
3578: Gladiator (2000)                              8.278976
1198: Raiders of the Lost Ark (1981)               -2.250455
1: Toy Story (1995)                                -8.733699
318: Shawshank Redemption, The (1994)              -9.806292
593: Silence of the Lambs, The (1991)             -12.818217
1259: Stand by Me (1986)                          -15.013091
780: Independence Day (ID4) (1996)                -24.566080
260: Star Wars: Episode IV - A New Hope (1977)    -27.179263
2916: Total Recall (1990)                        -203.454441
dtype: float64